In [2]:
import pandas as pd
import numpy as np
from predictables.core.src._UnivariateAnalysis import UnivariateAnalysis
from predictables.util import to_pl_lf
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset
bcancer = load_breast_cancer()
cancery = pd.Series(bcancer.target, name="y").map({0: "malignant", 1: "benign"})
cancerdf = pd.DataFrame(
    bcancer.data, columns=[c.replace(" ", "_") for c in bcancer.feature_names]
)

# Standardize the data
scaler = StandardScaler()
cancerdf = pd.DataFrame(scaler.fit_transform(cancerdf), columns=cancerdf.columns)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    cancerdf, cancery, test_size=0.2, random_state=42, stratify=cancery
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

# Perform PCA
pca = PCA(n_components=2)
X_train_pca = pd.DataFrame(
    pca.fit_transform(X_train), columns=["pc1", "pc2"], index=X_train.index
)
X_val_pca = pd.DataFrame(
    pca.transform(X_val), columns=["pc1", "pc2"], index=X_val.index
)
X_test_pca = pd.DataFrame(
    pca.transform(X_test), columns=["pc1", "pc2"], index=X_test.index
)

# Combine the data
df_train = pd.concat([X_train, X_train_pca], axis=1)
df_val = pd.concat([X_val, X_val_pca], axis=1)
df_test = pd.concat([X_test, X_test_pca], axis=1)

# Add the target variable
df_train["y"] = y_train.map({"malignant": 0, "benign": 1}).values
df_val["y"] = y_val.map({"malignant": 0, "benign": 1}).values
df_test["y"] = y_test.map({"malignant": 0, "benign": 1}).values

# Randomly sort training data into 5 cross-validation folds
df_train["fold"] = np.random.choice(range(5), size=df_train.shape[0]) + 1

df_train.to_parquet("cancer_train.parquet")
df_val.to_parquet("cancer_val.parquet")
df_test.to_parquet("cancer_test.parquet")

df_train = to_pl_lf(df_train)
df_val = to_pl_lf(df_val)
df_test = to_pl_lf(df_test)

df_train.head().collect()

mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,radius_error,texture_error,perimeter_error,area_error,smoothness_error,compactness_error,concavity_error,concave_points_error,symmetry_error,fractal_dimension_error,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,pc1,pc2,y,fold
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
0.2081,0.912292,0.347273,0.046959,0.57215,1.774977,1.015706,1.02817,-0.272428,0.55604,-0.453216,-0.462029,0.358868,-0.333042,0.346688,1.438701,0.783046,1.140961,0.594217,0.195592,-0.039178,0.342424,0.337735,-0.168554,-0.033692,1.339296,0.895753,0.884571,0.160555,0.169804,2.601883,1.664731,0,3
-1.684571,-0.57005,-1.658278,-1.288347,-0.737294,-0.85113,-0.9155,-1.109197,-0.155598,0.316465,-0.898232,-0.472008,-0.877224,-0.706961,0.642366,-0.50402,-0.530967,-0.953653,0.62933,-0.458783,-1.512777,-0.605327,-1.489328,-1.122222,-0.11698,-0.754239,-0.975761,-1.354653,0.330422,-0.546168,-4.564517,1.565109,1,2
-0.825712,0.132725,-0.825,-0.761051,0.643316,-0.692695,-1.052023,-1.066224,0.468713,-0.356897,-0.38825,1.35921,-0.449022,-0.455811,1.949753,-0.806941,-0.948182,-1.107752,2.65013,-0.69292,-0.888216,0.016737,-0.904036,-0.781363,0.439736,-1.002397,-1.241784,-1.437181,0.632947,-1.037706,-3.127716,1.357112,1,2
-0.169639,-1.943019,-0.167192,-0.27215,2.329937,0.006804,-0.251467,0.429234,2.1591,0.512094,0.017786,-0.368046,-0.105966,-0.169129,2.11976,0.162743,-0.672216,-0.577002,0.626908,0.896114,-0.453343,-2.147457,-0.473631,-0.483572,0.558093,-0.740244,-0.89617,-0.617229,-0.308601,-0.666975,-0.807603,2.071959,1,1
-0.215082,-0.674768,-0.241747,-0.288361,-1.794101,-0.58922,-0.098925,-0.539588,-1.422476,-0.647506,-0.870802,-0.139439,-0.813365,-0.56494,-0.374008,0.403626,0.586093,-0.229711,-1.024602,0.106325,-0.416068,-0.47668,-0.454866,-0.436812,-1.309316,-0.007411,0.28119,-0.378019,-1.379572,-0.424808,-2.212381,-0.936765,1,1


In [2]:
df_train.head()

<LazyFrame [34 cols, {"mean_radius": Float64 … "fold": Int64}] at 0x7FDB24F75FD0>

In [3]:
df_train.columns[:5] + df_train.columns[-4:]

['mean_radius',
 'mean_texture',
 'mean_perimeter',
 'mean_area',
 'mean_smoothness',
 'PC1',
 'PC2',
 'y',
 'fold']

In [4]:
ua = UnivariateAnalysis(
    model_name="Cancer Model",
    df_train=df_train,
    df_val=df_val,
    target_column_name="y",
    feature_column_names=df_train.columns[:5] + df_train.columns[-4:-2],
    cv_column_name="fold",
    time_series_validation=False,
)

Performing univariate analysis on 7 features:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
dir(ua)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_to_report',
 '_build_desc',
 '_feature_list',
 '_generate_segment_report',
 '_get_file_stem',
 '_rpt_filename',
 '_rpt_methodology_page',
 '_rpt_overview_page',
 '_rpt_title_page',
 '_segment_features',
 '_sort_features_by_ua',
 'build_report',
 'cv_column_name',
 'cv_folds',
 'df',
 'df_val',
 'feature_column_names',
 'get_features',
 'mean_area',
 'mean_perimeter',
 'mean_radius',
 'mean_smoothness',
 'mean_texture',
 'model_name',
 'pc1',
 'pc2',
 'target_column_name',
 'time_series_validation']

In [6]:
ua.build_report("cancer.pdf", max_per_file=20)

Building 7 univariate analysis reports:   0%|          | 0/7 [00:00<?, ?it/s]



idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)


idx.shape: (341,)
fold.drop_duplicates().sort_values().values: [1 2 3 4 5] | ROC000Fe
f: 1 | type(y): <class 'pandas.core.series.Series'> | type(yhat_proba): <class 'pandas.core.series.Series'> | type(fold): <class 'pandas.core.series.Series'> | type(time_series_validation): <class 'bool'> | type(n_bins): <class 'int'> | ROC_0001f
y.shape: (341,) | yhat_proba.shape: (341,) | fold.shape: (341,) | n_bins: 200 | ROC_0001g


idx.shape: (341,)


idx.shape: (341,)
f: 2 | type(y): <class 'pandas.core.series.Series'> | type(yhat_proba): <class 'pandas.core.series.Series'> | type(fold): <class 'pandas.core.series.Series